# openEO Platform - Sentinel-2 ARD Data Example
#### Sentinel-2 ARD retrieval

Currently, openEO Platform is only open for Early Adopters or within a free 30 day trial period.

To express your interest in becoming an Early Adopter you need to follow 2 steps:

1 - Connect an existing account to EGI check-in 2 - Apply to the openEO Platform virtual organization

The 2 steps are described in detail here: https://docs.openeo.cloud/join/early_adopter.html#preamble-registration-and-login-authentication

This notebook is based on the openEOPlatform github, sample-notebooks, here: https://github.com/openEOPlatform/sample-notebooks/blob/main/openEO%20Platform%20-%20Optical%20ARD.ipynb

### Import Libraries

Note: eo_utils.py must be in the same folder as the notebook. It can be retreived from the openEO github, here:
https://github.com/openEOPlatform/sample-notebooks

In [4]:
import openeo
import time
from datetime import datetime
import csv
from csv import writer
from eo_utils import *
from netCDF4 import Dataset
import numpy as np
import xarray as xr
import pandas as pd

In [23]:
openeo.__version__

'0.10.1a1'

### 1. Open the connection with openEO Platform. You can autenticate using EGI.

Please refer to the openEO Platform official documentation for details about the installation and authentication: https://docs.openeo.cloud/getting-started/python/

In [24]:
backend = "https://openeo.cloud"
conn = openeo.connect(backend)
conn.list_collections()
conn = openeo.connect(backend).authenticate_oidc(provider_id='egi')

Authenticated using refresh token.


In [25]:
backend = "openeo.cloud"
conn = openeo.connect(backend).authenticate_oidc(provider_id="egi")

Authenticated using refresh token.


### 2. Area of Interest selection

Defining Area of Interest (AOI) per centre coordinates and zoom level.
If necessary, draw and adjust the bounding box (bbox) over the map and get its coordinates with the getBbox command.

In [26]:
center = [32.72, -17.19]
zoom = 12

eoMap = openeoMap(center,zoom)
eoMap.map

Map(center=[32.72, -17.19], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [27]:
bbox = eoMap.getBbox()
print('west',bbox[0],'\neast',bbox[2],'\nsouth',bbox[1],'\nnorth',bbox[3])

IndexError: tuple index out of range

### 3. Define range of interest in time and requested bands from Sentinel-2 ARD data

In [28]:
conn.describe_collection('TERRASCOPE_S2_TOC_V2')

{'assets': {'metadata_iso_19139': {'href': 'https://services.terrascope.be/collectioncatalogue/srv/api/records/urn:eop:VITO:TERRASCOPE_S2_TOC_V2/formatters/xml?approved=true',
   'roles': ['metadata', 'iso-19139'],
   'title': 'ISO 19139 metadata',
   'type': 'application/vnd.iso.19139+xml'},
  'thumbnail': {'href': 'https://services.terrascope.be/collectioncatalogue/images/quicklooks/SENTINEL2_TOC_V2_QL.JPG',
   'roles': ['thumbnail'],
   'title': 'Thumbnail',
   'type': 'image/jpeg'}},
 'common_name': 'SENTINEL2_L2A',
 'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['B01',
    'B02',
    'B03',
    'B04',
    'B05',
    'B06',
    'B07',
    'B08',
    'B8A',
    'B11',
    'B12',
    'SCL',
    'relativeAzimuthAngles',
    'sunZenithAngles',
    'viewZenithAngles']},
  't': {'extent': ['2015-07-06T00:00:00Z', None], 'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-180, 180],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 10,
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-56, 83],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 10,
   'type': 'spatial'}},
 'description': '[Sentinel-2 Level 2A](https://docs.terrascope.be/#/DataProducts/Sentinel-2/Level2A/Level2A) product, processed from L1C by Sen2Cor. \n Use the [Terrascope viewer](https://viewer.terrascope.be/?language=nl&bbox=0.9339867511378304,50.14520994379217,6.207424251137831,51.77292299745045&overlay=false&bgLayer=MapBox&date=2019-02-25T03:17:15.000Z&layer=CGS_S2_RADIOMETRY) to explore the data. \n\n ',
 'extent': {'spatial': {'bbox': [[-180, -56, 180, 83]]},
  'temporal': {'interval': [['2015-07-06T00:00:00Z', None]]}},
 'id': 'TERRASCOPE_S2_TOC_V2',
 'keywords': ['VITO',
  'TERRASCOPE',
  'COPERNICUS',
  'ESA',
  'Orthoimagery',
  'SENTINEL-2',
  'MSI',
  'Level-2A',
  'Radiometry',
  'Plant Resource',
  'TOC'],
 'license': 'proprietary',
 'links': [{'href': 'https://sentinel.esa.int/documents/247904/690755/Sentinel_Data_Legal_Notice',
   'rel': 'license'},
  {'href': 'https://services.terrascope.be/catalogue/description?collection=urn:eop:VITO:TERRASCOPE_S2_TOC_V2',
   'rel': 'alternate'},
  {'href': 'https://viewer.terrascope.be/?language=nl&bbox=0.9339867511378304,50.14520994379217,6.207424251137831,51.77292299745045&overlay=false&bgLayer=

In [29]:
collection      = 'TERRASCOPE_S2_TOC_V2'
spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3],'crs':'EPSG:4326'}
temporal_extent = ["2017-09-01", "2022-03-01"]
bands           = ["B02","B03","B04","B05","B06","B07","B08",'B8A','B11',"B12"]

### 4. Save process as final step, we use NetCDF as output format

In [30]:
s2_l2a = conn.load_collection(collection,spatial_extent=spatial_extent,bands=bands,temporal_extent=temporal_extent)

s2_l2a_nc = s2_l2a.save_result(format="NETCDF")
print(s2_l2a_nc)

DataCube(<PGNode 'save_result' at 0x7f2b98e33590>)


 ### 5. Create a batch job and start it. More info about batch jobs can be found here:

https://docs.openeo.cloud/getting-started/python/#batch-job-execution

https://open-eo.github.io/openeo-python-client/batch_jobs.html

In [31]:
job = conn.create_job(s2_l2a_nc)
job_id = job.job_id
if job_id:
    print("Batch job created with id: ",job_id)
    job.start_job()
else:
    print("Error! Job ID is None")

Batch job created with id:  vito-cdcd9cee-fd2b-4565-bbd4-3849fc1de58a


### 8. Get a description of the job and check its status.

In [33]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S.%f)")

my_job = conn.job(job_id)
job_description = my_job.describe_job()

header = ['job_description', 'job_timestamp', 'job_id','job_status']
data = ['marismar_s2l2_optical_2017_2022_timeseries',timestampStr, job_id,job_description['status']]

with open('job_id.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write the data
    writer.writerow(data)
    

    
print("Batch job with id: ",job_id, ' is ',job_description['status'])

Batch job with id:  vito-cdcd9cee-fd2b-4565-bbd4-3849fc1de58a  is  running


In [35]:
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S.%f)")

my_job = conn.job(job_id)
job_description = my_job.describe_job()

header = ['job_description', 'job_timestamp', 'job_id','job_status']
data = ['marismar_s2l2_optical_2017_2022_timeseries',timestampStr, job_id,job_description['status']]

with open('job_id.csv', 'a') as f_object:

    writer_object = csv.writer(f_object)
  
    writer_object.writerow(data)
  
    #Close the file object
    f_object.close()
    
print("Batch job with id: ",job_id, ' is ',job_description['status'])

Batch job with id:  vito-cdcd9cee-fd2b-4565-bbd4-3849fc1de58a  is  finished


### 9. Download and visualize the results

In [36]:
%%time

results = my_job.get_results()
metadata = results.get_metadata()
results.download_file("./marismar_s2l2_optical_2017_2022_timeseries.nc")

CPU times: user 16.9 s, sys: 35.2 s, total: 52.1 s
Wall time: 1h 27min 28s


PosixPath('marismar_s2l2_optical_2017_2022_timeseries.nc')

### 10. Extract CSV time series for single reference lat/lon

In [2]:
lat_ref = 32.72
lon_ref = -17.19
dataset = xr.open_dataset(r'./marismar_s2l2_optical_2017_2022_timeseries.nc')
subset = dataset.sel(x=lon_ref, y=lat_ref, method='nearest')
subset

<xarray.Dataset>
Dimensions:  (t: 147)
Coordinates:
  * t        (t) datetime64[ns] 2019-10-01 2019-10-26 ... 2022-02-22 2022-02-27
    x        float64 2.818e+05
    y        float64 3.614e+06
Data variables:
    crs      |S1 ...
    B02      (t) float32 ...
    B03      (t) float32 ...
    B04      (t) float32 ...
    B05      (t) float32 ...
    B06      (t) float32 ...
    B07      (t) float32 ...
    B08      (t) float32 ...
    B8A      (t) float32 ...
    B11      (t) float32 ...
    B12      (t) float32 ...
Attributes:
    Conventions:  CF-1.8
    institution:  openEO platform - Geotrellis backend: 0.6.3a1
    description:  
    title:

In [5]:
all_dates = dataset.t.values
sel_dates = [date for date in all_dates if pd.to_datetime(date) >= pd.to_datetime("01/09/2017")] #select dates past 1st Jan 1948
result = subset.sel(t=sel_dates)
print(result)

<xarray.Dataset>
Dimensions:  (t: 147)
Coordinates:
  * t        (t) datetime64[ns] 2019-10-01 2019-10-26 ... 2022-02-22 2022-02-27
    x        float64 2.818e+05
    y        float64 3.614e+06
Data variables:
    crs      |S1 b''
    B02      (t) float32 nan nan 190.0 222.0 nan ... 52.0 224.0 129.0 441.0 90.0
    B03      (t) float32 nan nan 71.0 143.0 nan ... -27.0 205.0 59.0 446.0 23.0
    B04      (t) float32 nan nan 26.0 79.0 nan ... -18.0 221.0 18.0 449.0 24.0
    B05      (t) float32 nan nan 34.0 76.0 nan ... -26.0 199.0 11.0 482.0 12.0
    B06      (t) float32 nan nan 39.0 61.0 nan ... -25.0 188.0 15.0 439.0 4.0
    B07      (t) float32 nan nan 27.0 76.0 nan ... -28.0 181.0 16.0 447.0 13.0
    B08      (t) float32 nan nan 38.0 80.0 nan ... 8.0 150.0 3.0 390.0 -2.0
    B8A      (t) float32 nan nan 25.0 79.0 nan ... -3.0 159.0 10.0 437.0 26.0
    B11      (t) float32 nan nan 46.0 76.0 nan ... 62.0 206.0 31.0 377.0 47.0
    B12      (t) float32 nan nan 40.0 69.0 nan ... 72.0 194.0

In [6]:
result.to_pandas().to_csv('marismar_s2l2_optical_2017_2022_timeseries.csv')